This notebook is for creating the functionality of querying an external database and extracting data from that database, and storing it into a numpy array

In [13]:
#import the needed packages
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from bs4.dammit import EncodingDetector
import requests
import re

In [18]:
#define the needed functions

def make_web_address(dspacing1, dspacing2):
    '''
    make_web_address will compile a string to use for a webpage address
    
    For example, if I want to search for rutile on the American 
    Minerals Society webpage, I might use this address:
    
    http://rruff.geo.arizona.edu/AMS/result.php?diff=vals(3.2435,2.4836),opt(),type(d-spacing),tolerance(.001)
    
    Where 3.2435 and 2.4836 are two d spacings in the rutile structure
    '''
    
    web_address = 'http://rruff.geo.arizona.edu/AMS/result.php?diff=vals(' + str(dspacing1) + ',' + str(dspacing2) +'),opt(),type(d-spacing),tolerance(.001)'
    
    return web_address

def find_diffraction_files(href):
    '''
    find_diffraction_files is a sorting function that can be passed 
    into compile_links to generate a specific set of diffraction text 
    file web addresses from the American Mineral Society database
    '''
#    return href and re.compile("txt").search(href) and not re.compile("dif").search(href)        
    return href and re.compile("dif").search(href)        

def compile_links(web_address, links_array):
    '''
    access_web accesses a webpage at a given address,
    finds all of the links on that page, and appends those links 
    to a list called links.
    
    inputs are a web address, and the array for storing links
    '''
    
    html_page = requests.get(web_address)
    http_encoding = html_page.encoding if 'charset' in html_page.headers.get('content-type', '').lower() else None
    html_encoding = EncodingDetector.find_declared_encoding(html_page.content, is_html=True)
    encoding = html_encoding or http_encoding
    soup = BeautifulSoup(html_page.content, from_encoding=encoding)
    
    for link in soup.find_all(href=find_diffraction_files):
        links_array.append(link['href'])
    
    return links_array

def get_diffraction_data(links_array, dspacing1, dspacing2, pot_planes_df):
    '''
    get_diffraction_data will compile crystal structure parameters and 
    indexes of all potential planes into a pandas DataFrame
    
    Inputs are an array of truncated href, two d spacings from the FFT, 
    and the pandas DataFrame into which data will be stored. Returns the
    populated DataFrame.
    '''

    for entry in links_array:
        full_address = 'http://rruff.geo.arizona.edu' + entry
#        dif_csv = requests.get(full_address)
#        soup = BeautifulSoup(dif_csv.content)
        temp_df = pd.read_csv(full_address)
        print(temp_df)
       

    return pot_planes_df
    


generate a web search of the minerals with matching d spacings

In [15]:
dspacing1 = 3.3
dspacing2 = 3.5

make_web_address(dspacing1, dspacing2)


'http://rruff.geo.arizona.edu/AMS/result.php?diff=vals(3.3,3.5),opt(),type(d-spacing),tolerance(.001)'

create a list of links to diffraction text files for matching mineral structures

In [16]:
web_address = 'http://rruff.geo.arizona.edu/AMS/result.php?diff=vals(3.3,3.5),opt(),type(d-spacing),tolerance(.001)'
links_array = []
compile_links(web_address, links_array)

['/AMS/download.php?id=01755.txt&down=dif',
 '/AMS/download.php?id=16200.txt&down=dif',
 '/AMS/download.php?id=17097.txt&down=dif',
 '/AMS/download.php?id=18445.txt&down=dif',
 '/AMS/download.php?id=20133.txt&down=dif']

In [19]:
dspacing1 = 3.3
dspacing2 = 3.5

column_names = ["mineral", "symetry", "a", "b", "c", "alpha", "beta", "gama", "dspacing", "h", "k", "l"]
pot_planes_df = pd.DataFrame(columns = column_names)

get_diffraction_data(links_array, dspacing1, dspacing2, pot_planes_df)

ParserError: Error tokenizing data. C error: Expected 3 fields in line 4, saw 4
